In [1]:
import ROOT
import glob

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x493d0a0


In [2]:
%jsroot on

In [3]:
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
%%cpp
using namespace ROOT::VecOps;

In [5]:
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [6]:
df = ROOT.RDataFrame("events", "data/gen/simple_whizard_rescan/enuqq_rescan.edm4hep.root")
paths = glob.glob("/eos/user/l/lreichen/paris/*.edm4hep.root")
files = [f"root://eosuser.cern.ch/{path}" for path in paths]
df = ROOT.RDataFrame("events", files)
# df = ROOT.RDataFrame("events", "root://eosuser.cern.ch//eos/user/l/lreichen/paris/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500105.P4f_sw_sl.eL.pL.n000.d_dstm_15064_0.miniDST.edm4hep.root")

In [7]:
df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPDoubleKeys, GPDoubleValues); par.loadFrom(GPFloatKeys, GPFloatValues); par.loadFrom(GPIntKeys, GPIntValues); par.loadFrom(GPStringKeys, GPStringValues); return par;")

In [8]:
float_params = ["Energy", "WWCategorisation.mInv", "WWCategorisation.missE", "WWCategorisation.misspT", "beamPol1", "beamPol2", "crossSection", "crossSectionError"]
int_params = ["WWCategorisation.RecoCatAdvanced", "WWCategorisation.RecoCatBasic", "WWCategorisation.TrueCat", "WWCategorisation.nPFO", "beamPDG1", "beamPDG2"]

In [9]:
def parse_par(par):
    return par.replace(".", "_")

In [10]:
for fp in float_params:
    df = df.Define(f"params_{parse_par(fp)}", f"Parameters.get<float>(\"{fp}\").value()")
for ip in int_params:
    df = df.Define(f"params_{parse_par(ip)}", f"Parameters.get<int>(\"{ip}\").value()")

In [11]:
df.Snapshot("events", "data/paris/nano/all_params_flat.root", [f"params_{parse_par(p)}" for p in float_params + int_params])

<cppyy.gbl.ROOT.RDF.RResultPtr<ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void> > object at 0xdd95ee0>